In [10]:
from progpy.models import NEW_BatteryElectroChemEODEOL

batt = NEW_BatteryElectroChemEODEOL()
batt2 = NEW_BatteryElectroChemEODEOL()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('states:', batt.states)
print('params:', batt.parameters)

print('\n\ninputs:', batt2.inputs)
print('outputs:', batt2.outputs)
print('event(s): ', batt2.events)
print('states:', batt2.states)
print('params:', batt2.parameters)

config = {
    'save_freq': 1000,
    'dt': 2,
    'events': 'InsufficientCapacity',
    'print': True
}

def future_loading1(t, x=None):
    load = 1
    
    if x is not None:
        event_state = batt.event_state(x)
        if event_state["EOD"] > 0.95:
            load = 1 # Discharge
        elif event_state["EOD"] < 0.05:
            load = -1 # Charge
            
    return batt.InputContainer({'i': load})

config2 = {
    'save_freq': 1000,
    'dt': 2,
    'events': 'InsufficientCapacity',
    'print': False
}

def future_loading2(t, x=None):
    load = 1
    
    if x is not None:
        event_state = batt2.event_state(x)
        if event_state["EOD"] > 0.95:
            load = 1 # Discharge
        elif event_state["EOD"] < 0.05:
            load = -1 # Charge

    return batt2.InputContainer({'i': load})

inputs: ['i']
outputs: ['t', 'v']
event(s):  ['EOD', 'InsufficientCapacity']
states: ['tb', 'Vo', 'Vsn', 'Vsp', 'qnB', 'qnS', 'qpB', 'qpS', 'qMobile', 'tDiffusion', 'Ro', 'qMax', 'D']
params: {'process_noise': {'tb': np.float64(0.0), 'Vo': np.float64(0.0), 'Vsn': np.float64(0.0), 'Vsp': np.float64(0.0), 'qnB': np.float64(0.0), 'qnS': np.float64(0.0), 'qpB': np.float64(0.0), 'qpS': np.float64(0.0), 'qMobile': np.float64(0.0), 'tDiffusion': np.float64(0.0), 'Ro': np.float64(0.0), 'qMax': np.float64(0.0), 'D': np.float64(0.0)}, 'measurement_noise': {'t': np.float64(0.0), 'v': np.float64(0.0)}, 'xnMax': 0.6, 'xnMin': 0.0, 'xpMax': 1.0, 'xpMin': 0.4, 'qMax': 12666.666666666668, 'wq': -0.01, 'wr': 1e-06, 'wd': 0.01, 'qMaxThreshold': 5320, 'alpha': 0.5, 'Sn': 0.000437545, 'Sp': 0.00030962, 'kn': 2120.96, 'kp': 248898, 'Vol': 2e-05, 'VolSFraction': 0.1, 'to': 6.08671, 'tsn': 1001.38, 'tsp': 46.4311, 'U0p': 4.03, 'Ap': [-31593.7, 0.106747, 24606.4, -78561.9, 13317.9, 307387, 84916.1, -1074690.0

In [11]:
result = batt.simulate_to_threshold(future_loading1, **config)

Time: 0.0
	Input: {'i': np.float64(1.0)}
	State: {'tb': np.float64(292.1), 'Vo': np.float64(0.0), 'Vsn': np.float64(0.0), 'Vsp': np.float64(0.0), 'qnB': np.float64(6840.0), 'qnS': np.float64(760.0), 'qpB': np.float64(4560.000000000001), 'qpS': np.float64(506.6666666666668), 'qMobile': np.float64(7600.0), 'tDiffusion': np.float64(7000000.0), 'Ro': np.float64(0.117215), 'qMax': np.float64(7600.0), 'D': np.float64(7000000.0)}
	Output: {'t': np.float64(18.950000000000045), 'v': np.float64(4.191350293826752)}
	Event State: {'EOD': np.float64(1.0), 'InsufficientCapacity': np.float64(1.0)}

Time: 1000.0
	Input: {'i': np.float64(1.0)}
	State: {'tb': np.float64(292.49619956160086), 'Vo': np.float64(0.11820891329000098), 'Vsn': np.float64(0.029909158526808935), 'Vsp': np.float64(0.000654425321375614), 'qnB': np.float64(5951.340015995771), 'qnS': np.float64(648.6599840041091), 'qpB': np.float64(5448.659984004231), 'qpS': np.float64(618.006682662558), 'qMobile': np.float64(7590.019999999782), 'tDi

In [12]:
result2 = batt2.simulate_to_threshold(future_loading2, **config2)

In [ ]:
print(result.times == result2.times)
print(result.inputs == result2.inputs)
print(result.states == result2.states)
# Slightly off
print(result.event_states == result2.event_states)
print(result.outputs == result2.outputs)

True
True
True
False
True


In [21]:
eod2 = [event_state['EOD'] for event_state in result2.event_states]
eod = [event_state['EOD'] for event_state in result.event_states]
print(eod2 == eod)

False


In [22]:
ic2 = [event_state['InsufficientCapacity'] for event_state in result2.event_states]
ic = [event_state['InsufficientCapacity'] for event_state in result.event_states]
print(ic == ic2)

True


In [24]:
for i in range(len(result.times)):
    print(result.event_states[i]['EOD'], result2.event_states[i]['EOD'])

1.0 1.0
0.8695652173913135 0.8695629260529049
0.7387862796833791 0.7387843303843542
0.6076618229854556 0.6076602175423444
0.4761904761904714 0.47618921643063955
0.3443708609271506 0.3443699486888759
0.21220159151192447 0.21220102864447446
0.07968127490036588 0.07968106326407566
0.04999999999996287 0.049999867021593127
0.050066577896102776 0.05006644456336092
0.050133333333299085 0.050133199644766695
0.05020026702266414 0.0502001329769152
0.05026737967911306 0.05026724527471392
0.04979919678711849 0.04979906345629263
0.04986595174259826 0.049865818054078005
0.049932885906012664 0.04993275185835667
0.04999999999997384 0.049999865591733
0.050067294750984696 0.05006715998070211
0.05013477088946463 0.050134635755675526
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.07039563437926931 0.07039544230439264
0.07267759562842535 0.07267739705630225
0.07496580027361202 0.07496559516938446
0.07726027397262121 0.07726006230121764
0.07215606834242827 0.07335981935208125
0.0600365908